In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from skimage import data

tf.__version__

'2.1.0'

# Convolutions

## One-dimensional convolutions

In [4]:
array = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])
kernel = np.array([1, -1, 0])
conv = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Output array
for ii in range(8):
    conv[ii] = (kernel * array[ii:ii+len(kernel)]).sum()

# Print conv
print(conv)

[ 1 -1  1 -1  1 -1  1 -1  0  0]


## Defining image convolution kernels

In [13]:
# define a kernel that finds horizontal lines in images
kernel = np.array([[-1, -1, -1], 
                   [1, 1, 1],
                   [-1, -1, -1]])

# define a kernel that finds a light spot surrounded by dark pixels
kernel = np.array([[-1, -1, -1], 
                   [-1, 1, -1],
                   [-1, -1, -1]])

# define a kernel that finds a dark spot surrounded by light pixels
kernel = np.array([[1, 1, 1], 
                   [1, -1, 1],
                   [1, 1, 1]])

# Implementing image convolutions in `keras`

In [15]:
(_, _), (X, y) = tf.keras.datasets.fashion_mnist.load_data()

In [17]:
X = np.expand_dims(X, -1)
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.1,
                                                    stratify=y,
                                                    random_state=42)

## Convolutional network for image classification

In [39]:
# define model arch
model = Sequential()

model.add(Conv2D(20, kernel_size=3, input_shape=(28, 28, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

## Training a CNN to classify clothing types

In [42]:
# compile
model.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
# train
early_stop = EarlyStopping(monitor='val_loss',
                           patience=3)

model.fit(X_train, y_train,
          validation_split=0.2,
          batch_size=256,
          epochs=100,
          callbacks=[early_stop],
          verbose=2)

Train on 7200 samples, validate on 1800 samples
Epoch 1/100
7200/7200 - 1s - loss: 0.1722 - accuracy: 0.9408 - val_loss: 1.0195 - val_accuracy: 0.8267
Epoch 2/100
7200/7200 - 1s - loss: 0.0919 - accuracy: 0.9669 - val_loss: 1.0413 - val_accuracy: 0.8433
Epoch 3/100
7200/7200 - 1s - loss: 0.0532 - accuracy: 0.9811 - val_loss: 1.0124 - val_accuracy: 0.8417
Epoch 4/100
7200/7200 - 1s - loss: 0.0439 - accuracy: 0.9858 - val_loss: 1.0069 - val_accuracy: 0.8489
Epoch 5/100
7200/7200 - 1s - loss: 0.0285 - accuracy: 0.9915 - val_loss: 1.0097 - val_accuracy: 0.8522
Epoch 6/100
7200/7200 - 1s - loss: 0.0601 - accuracy: 0.9806 - val_loss: 1.0494 - val_accuracy: 0.8317
Epoch 7/100
7200/7200 - 1s - loss: 0.0335 - accuracy: 0.9896 - val_loss: 1.0637 - val_accuracy: 0.8511


In [44]:
print(f'test acc: {model.evaluate(X_test, y_test)[1]: .1%}')

1000/1000 [==============================] - 1s 534us/sample - loss: 1.0344 - accuracy: 0.8500
test acc:  85.0%


# Tweaking your convolutions

## Add padding to a CNN

In [45]:
# define model arch
model = Sequential()

model.add(Conv2D(20, kernel_size=3, input_shape=(28, 28, 1), padding='same', activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

## Add strides to a convolutional network

In [46]:
# define model arch
model = Sequential()

model.add(Conv2D(20, kernel_size=3, input_shape=(28, 28, 1), strides=2, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

To calculate the output dim of a convolution:
$$
Output = ((I - K + 2P)/S) + 1
$$
where

$I$: Input shape 

$K$: Kernel size 

$P$: Padding 

$S$: Stride